In [10]:
%pip install --upgrade transformers -q
%pip install --upgrade torch accelerate -q
%pip install bitsandbytes -q
%pip install auto-gptq -q
%pip install unsloth-zoo -q

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [23]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from unsloth import FastLanguageModel
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

In [2]:
model_name = "mistralai/Mistral-7B-v0.1"
max_seq_length = 2048  # Adjust as necessary

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length
)

==((====))==  Unsloth 2024.11.9: Fast Mistral patching. Transformers = 4.46.3.
   \\   /|    GPU: NVIDIA A10. Max memory: 21.975 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.1+cu118. CUDA = 8.6. CUDA Toolkit = 11.8.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [ ]:
FastLanguageModel.for_inference(model)

prompt = "What are the societal impacts of digital technology?"

inputs = tokenizer(prompt, return_tensors="pt")

output = model.generate(**inputs, max_new_tokens=100)
response = tokenizer.decode(output[0], skip_special_tokens=True)
print(response[len(prompt):].strip())

Add lora adapters

In [9]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    random_state = 1337,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.11.9 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [10]:
jsonl_file = "domain.jsonl"

Load and reformat the domain data.

In [15]:
# use the dataset loader by Huggingface and some formatting functions
dataset = load_dataset("json", data_files=jsonl_file, split="train")

tokenizer.pad_token = tokenizer.eos_token

def format_text(examples):
    texts = [note + tokenizer.pad_token for note in examples["text"]]
    return {"text": texts}

dataset = dataset.map(format_text, batched=True)

Tokenize the domain data with the pre-trained model's tokenizer.

In [16]:
# Initialize the tokenizer
#tokenizer = AutoTokenizer.from_pretrained(model_name)

# Tokenize the text
def tokenize_texts(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=512)

tokenized_dataset = dataset.map(tokenize_texts, batched=True)

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

## 4. Fine-tune the model

We want the *training loss* to decrease. A loss value around 2-3 is reasonable, if it gets close to 1.0 or drops below, the predictions will be highly confident, but also with some risk of overfitting, meaning that the model has learned the training data too well and may not perform as effectively on unseen data.

*See `README.md` for details about which parameters to tweak to avoid overfitting.*

In [17]:
# Remove unneeded columns and set format for PyTorch
tokenized_dataset = tokenized_dataset.remove_columns(["text"])  # Keep only tokenized columns
tokenized_dataset.set_format(type="torch")

In [24]:
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=tokenized_dataset,
    max_seq_length=1024,
    dataset_num_proc=2,
    packing=False,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        learning_rate=0.00001, # <<<<<<< THE HIGHER THE RATE THE FASTER TO OVERFIT
        warmup_steps=5,
        num_train_epochs=1,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.3,
        lr_scheduler_type="linear",
        seed=1337,
        output_dir="outputs",
        report_to="none",
    ),
)

trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 500 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 62
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,2.429900
2,2.726500
3,2.726400
4,2.732900
5,2.642800
6,2.909500
7,2.716400
8,2.405500
9,2.582200
10,2.391800


In [26]:
# Save the model and tokenizer after training
trainer.save_model("outputs")  # Saves the model, tokenizer, and training args
tokenizer.save_pretrained("outputs")

('outputs/tokenizer_config.json',
 'outputs/special_tokens_map.json',
 'outputs/tokenizer.model',
 'outputs/added_tokens.json',
 'outputs/tokenizer.json')

In [32]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Define the fine-tuned model path
fine_tuned_model_path = "outputs"

# Enable CPU offloading without unsupported arguments
model = AutoModelForCausalLM.from_pretrained(
    fine_tuned_model_path,
    max_memory = {"cuda:0": "10GB", "cpu": "8GB"},
    offload_folder="offload"  # Folder for offloaded layers on disk
)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(fine_tuned_model_path)

# Test the model with a prompt
prompt = "What are the societal impacts of digital technology?"
inputs = tokenizer(prompt, return_tensors="pt")
outputs = model.generate(**inputs, max_new_tokens=50)
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(response)

`low_cpu_mem_usage` was None, now default to True since model is quantized.


OutOfMemoryError: CUDA out of memory. Tried to allocate 28.00 MiB. GPU 0 has a total capacity of 21.98 GiB of which 21.12 MiB is free. Process 26660 has 14.09 GiB memory in use. Process 144218 has 7.85 GiB memory in use. Of the allocated memory 7.55 GiB is allocated by PyTorch, and 16.77 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)